By: Dominikus Krisna Herlambang | ©2022

Diffusion and advection are two fundamental processes that describe the movement of particles within a fluid or across a space. They may seem similar, but they have distinct characteristics:

## Diffusion

  - Process: Random movement of particles from areas of high concentration to areas of low concentration. Imagine tiny balls bouncing around in a box, spreading out over time.
  -Driving Force: Particles bumping into each other due to their inherent thermal energy.
  -Net Effect: Spreads and mixes particles over time, leading to a more uniform distribution.

## Advection

  - Process: Bulk movement of particles along with the flow of a fluid. Imagine riding a conveyor belt - you (the particle) are being carried along by the motion of the belt (the fluid).
  - Driving Force: External force, such as pressure gradients or wind, that causes the fluid to move.
  - Net Effect: Particles are transported in the direction of the flow, with little to no spreading.

## Together: Convection-Diffusion Equation

In many real-world scenarios, both diffusion and advection occur simultaneously. This combined phenomenon is described by the convection-diffusion equation, which is crucial in fields like fluid mechanics, heat transfer, and mass transport.

Understanding the relative importance of each process helps us predict the movement of substances in various contexts. Here's a simple analogy:

Imagine a drop of ink in a cup of still water (no advection). The ink will slowly spread outwards due to diffusion, eventually coloring the entire cup uniformly.

Now, imagine stirring the water (advection). The ink will be carried along with the flow, forming streaks and swirls instead of a uniform distribution.

The balance between diffusion and advection depends on a parameter called the Peclet number (Pe). If Pe is much greater than 1, advection dominates, and the substance is mainly transported by the flow. Conversely, if Pe is much less than 1, diffusion dominates, and the substance spreads out more evenly.

## This is an application of diffusion-advection: Modeling oil spills in waters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys

In [ ]:
def percentage(part, whole):
  percentage = 100 * float(part)/float(whole)
  return str(round(percentage,2)) + "X"

In [ ]:
# initial parameters
C = 0.49 # Flow Rate
ad = 1.49 # Diffusion Coefficient
# current direction (based on Geographical Convention)
theta = 49 # Scenario 1
# advanced parameters
q = 0.49 # Stability Criteria
x = 300 # Number of Horizontal Grids (x)
y = 300 # Number of Vertical Grids (y)
dx = 3
dy = 3
Tend = 104 # Simulation Time
#Tend = 1
dt = 0.5
# Pollutant
px = 150 # Pollutants on grid x
py = 134 # Pollutants on grid y
Ic = 594 # Amount of Pollutants
# U and V calculation ()
u = C * np.sin(theta*np.pi/180)
v = C * np.cos(theta*np.pi/180)
dt_count = 1/((abs(u)/(q*dx))+(abs(v)/(q*dy))+(2*ad/(q*dx*2))+(2*ad/(q*dy*2)))

In [ ]:
Nx = int(x/dx) # Number of meshes in x direction
Ny = int(y/dy) # Number of meshes in y direction
Nt = int(Tend/dt) # Number of time steps
# Pollutant Point Calculation at Discharge
px1 = int(px/dx)
py1 = int(py/dy)

# Simplified function
lx = u*dt/dx
ly = v*dt/dy
ax = ad*dt/dx**2
ay = ad*dt/dy**2
cfl = (2*ax + 2*ay + abs(lx) + abs(ly)) # CFL Stability Requirements

In [ ]:
# CFL Calculation
if cfl >= q:
  print('CFL Violated, Please use dt :' + str(round(dt_count,4)))
  sys.exit()
#%%
# Grid Creation
x_grid = np.linspace(0-dx, x+dx, Nx+2)
y_grid = np.linspace(0-dx, y+dy, Ny+2)
t = np.linspace(0, Tend, Nt+1)
x_mesh, y_mesh = np.meshgrid(x_grid,y_grid)
F = np.zeros((Nt+1, Ny+2, Nx+2))
# Initial Condition
F[0,py1,px1] = Ic
#%%
# Iteration
for n in range (0, Nt):
  for i in range (1,Ny+1):
    for j in range (1,Nx+1):
      F[n+1,i,j]=((F[n,i,j]*(1-abs(lx)-abs(ly)))  + \
                  (0.5*(F[n,i-1,j]*(ly+abs(ly)))) + \
                  (0.5*(F[n,i+1,j]*(abs(ly)-ly))) + \
                  (0.5*(F[n,i,j-1]*(lx+abs(lx)))) + \
                  (0.5*(F[n,i,j+1]*(abs(lx)-lx))) + \
                  (ay*(F[n,i+1,j]-2*(F[n,i,j])+F[n,i-1,j])) + \
                  (ay*(F[n,i,j+1]-2*(F[n,i,j])+F[n,i,j-1]))) # Discretization
  # Dirichlet Boundary Condition
  F[n+1,0,:] = 0 # BC Bottom
  F[n+1,:,0] = 0 # BC Left
  F[n+1,Ny+1,:] = 0 # BC Top
  F[n+1,:,Nx+1] = 0 # BC Right

  # Picture Output
  plt.clf()
  plt.pcolor(x_mesh, y_mesh, F[n+1,:,:], cmap='hot_r', edgecolor='k')
  cbar = plt.colorbar(orientation='vertical', shrink=0.95, extend='both')
  cbar.set_label(label='Concentration', size=8)
  #plt.clim(0,100)
  plt.title('Oil Spill Model\nt='+str(round(dt*(n+1),3))+', Initial Condition='+str(Ic))
  plt.xlabel('x_grid',fontsize=11)
  plt.ylabel('y_grid',fontsize=11)
  plt.axis([0,x,0,y])
  #plt.pause(0.01)
  plt.savefig(str(n+1)+'.png', dpi=300)
  plt.pause(0.01)
  plt.close()
  print('running timestep ke:' +str(n+1) + ' dari:' +str(Nt) + '('+ percentage(n+1,Nt)+')')
  #print('Nilai CFL:' +str(cfl) + ' dengan arah:' +str(theta))